# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
#from api_keys import g_key
g_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
weather_data = pd.read_csv("WeatherPy.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.29,-156.79,39.20,86,90,12.75,US,1598843119
1,atuona,-9.80,-139.03,78.96,79,38,25.99,PF,1598843113
2,conde,-7.26,-34.91,73.40,94,0,6.93,BR,1598843519
3,rikitea,-23.12,-134.97,69.46,82,27,17.07,PF,1598843107
4,hambantota,6.12,81.12,82.40,83,40,6.93,LK,1598843139
...,...,...,...,...,...,...,...,...,...
540,uhlove,44.82,33.60,71.49,43,0,5.19,UA,1598843647
541,warri,5.52,5.75,73.98,93,93,2.59,NG,1598843647
542,fuzhou,26.06,119.31,89.60,62,40,15.66,CN,1598843590
543,artyom,40.47,50.33,69.80,94,0,9.17,AZ,1598843648


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [18]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,conde,-7.26,-34.91,73.40,94,0,6.93,BR,1598843519
56,kurchum,48.57,83.65,70.23,36,0,2.55,KZ,1598843531
74,aksu,41.12,80.26,76.12,35,0,1.36,CN,1598843535
87,correntina,-13.34,-44.64,70.23,55,0,6.24,BR,1598843537
111,sakakah,29.97,40.21,78.80,14,0,5.82,SA,1598843343
145,jalu,29.03,21.55,77.77,33,0,7.76,LY,1598843551
148,derzhavinsk,51.10,66.32,72.68,40,0,7.47,KZ,1598843552
151,lagunas,-5.23,-75.68,78.44,70,0,1.68,PE,1598843553
183,amapa,1.00,-52.00,72.88,92,0,3.02,BR,1598843561
207,shache,38.42,77.24,75.20,23,0,3.15,CN,1598843181


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:

# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,conde,BR,-7.26,-34.91,
56,kurchum,KZ,48.57,83.65,
74,aksu,CN,41.12,80.26,
87,correntina,BR,-13.34,-44.64,
111,sakakah,SA,29.97,40.21,
145,jalu,LY,29.03,21.55,
148,derzhavinsk,KZ,51.10,66.32,
151,lagunas,PE,-5.23,-75.68,
183,amapa,BR,1.00,-52.00,
207,shache,CN,38.42,77.24,


In [21]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [22]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 2: conde.
Closest hotel in conde is POUSADA PORTO DO SOL.
------------
Retrieving Results for Index 56: kurchum.
Missing field/result... skipping.
------------
Retrieving Results for Index 74: aksu.
Closest hotel in aksu is Pudong Holiday Hotel.
------------
Retrieving Results for Index 87: correntina.
Closest hotel in correntina is Correntina Palace Hotel.
------------
Retrieving Results for Index 111: sakakah.
Closest hotel in sakakah is Raoum Inn Hotel.
------------
Retrieving Results for Index 145: jalu.
Closest hotel in jalu is Jalu Hotel.
------------
Retrieving Results for Index 148: derzhavinsk.
Closest hotel in derzhavinsk is Gostinitsa "Ishim".
------------
Retrieving Results for Index 151: lagunas.
Closest hotel in lagunas is Hospedaje Eco.
------------
Retrieving Results for Index 183: amapa.
Missing field/result... skipping.
------------
Retrieving Results for Index 207: shache.
Closest hotel in shache is Shache Hotel.
------------
Retrieving R

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,conde,BR,-7.26,-34.91,POUSADA PORTO DO SOL
56,kurchum,KZ,48.57,83.65,
74,aksu,CN,41.12,80.26,Pudong Holiday Hotel
87,correntina,BR,-13.34,-44.64,Correntina Palace Hotel
111,sakakah,SA,29.97,40.21,Raoum Inn Hotel
145,jalu,LY,29.03,21.55,Jalu Hotel
148,derzhavinsk,KZ,51.10,66.32,"Gostinitsa ""Ishim"""
151,lagunas,PE,-5.23,-75.68,Hospedaje Eco
183,amapa,BR,1.00,-52.00,
207,shache,CN,38.42,77.24,Shache Hotel


In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))